# Movie Recommendation System using NLP & Cosine Similarity

This project builds a content-based movie recommendation system using NLP techniques and cosine similarity.

The system recommends top 5 similar movies based on:
- Genres
- Keywords
- Cast
- Crew
- Overview

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem.porter import PorterStemmer
import pickle

## Data Loading

Dataset used: TMDB 5000 Movies Dataset

In [ ]:
credits=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')

In [ ]:
credits.head()

In [ ]:
movies.head()

In [ ]:
mov=movies.merge(credits, on='title')

In [ ]:
mov.head()

## Data Preprocessing

Will select relevant features and handle missing values.

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
mov.shape

In [ ]:
mov.info()

In [ ]:
mov=mov[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
mov.head()

In [ ]:
mov.info()

In [ ]:
mov.duplicated().sum()

## Feature Engineering

Extracting names from JSON columns and combining into a single 'tags' column.

In [ ]:
mov.iloc[0].genres

In [ ]:
import ast
ast.literal_eval

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
mov['genres']=mov['genres'].apply(convert)

In [ ]:
mov['genres']

In [ ]:
mov.iloc[0].keywords

In [ ]:
mov['keywords']=mov['keywords'].apply(convert)

In [ ]:
mov.head()

In [ ]:
mov.iloc[0].cast

In [ ]:
# Considering only top 3 names of crew & Director from cast

In [ ]:
def convert3(obj):
    L=[]
    c=0
    for i in ast.literal_eval(obj):
        if c!=3:
            L.append(i['name'])
            c=c+1
    return L

In [ ]:
mov['cast']=mov['cast'].apply(convert3)

In [ ]:
mov['cast']

In [ ]:
mov.iloc[0].crew

In [ ]:
def fetch_dirc(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [ ]:
mov['crew']=mov['crew'].apply(fetch_dirc)

In [ ]:
mov.head()

In [ ]:
mov['overview'][0]

In [ ]:
mov.head()

In [ ]:
mov['overview'] = mov['overview'].fillna('')
mov['overview'] = mov['overview'].apply(lambda x: x.split())

In [ ]:
mov.head()

In [ ]:
# replacing gap between words for ex- Science fiction --- ScienceFiction so that model do not consider tehm as different

In [ ]:
mov['genres'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
mov['genres']=mov['genres'].apply(lambda x: [i.replace(" ","") for i in x])
mov['keywords']=mov['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
mov['cast']=mov['cast'].apply(lambda x: [i.replace(" ","") for i in x])
mov['crew']=mov['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
mov.head()

In [ ]:
# Adding all the contents into 1 text column

In [ ]:
mov['tags1']=mov['overview']+mov['genres']+ mov['keywords']+mov['cast']+mov['crew']

In [ ]:
mov_rev=mov[['movie_id','title','tags1']]

In [ ]:
mov_rev

In [ ]:
mov_rev['tags1']=mov_rev['tags1'].apply(lambda x:" ".join(x))

In [ ]:
mov_rev

In [ ]:
mov_rev['tags1'][0]

In [ ]:
mov_rev['tags1'][1]

## Text Processing (Stemming)

Converting words into their base form

In [ ]:
import sys
!{sys.executable} -m pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
ps.stem('dancing')

In [ ]:
def stemmm(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
        

In [ ]:
stemmm('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d JamesCameron')

In [ ]:
mov_rev['tags1']=mov_rev['tags1'].apply(stemmm)

In [ ]:
mov_rev

## Vectorization using Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(mov_rev['tags1']).toarray()

In [ ]:
cv.get_feature_names_out()

In [ ]:
vectors.shape

## Cosine Similarity 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity[1]  # Shows similarity of movie 1 with other 4808 movies

In [ ]:
similarity.shape

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

## Recommendation Function to list similar 5 movies

In [ ]:
def recommend(movie):
    mov_index=mov_rev[mov_rev['title']==movie].index[0]
    distances= similarity[mov_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(mov_rev.iloc[i[0]].title) 

In [ ]:
recommend('Avatar')

## Save Model Files

In [ ]:
pickle.dump(mov_rev.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
top_5_similarities = {}

for i in range(len(similarity)):

    sim_scores = list(enumerate(similarity[i]))

    # remove self similarity
    sim_scores = [x for x in sim_scores if x[0] != i]

    sim_scores = sorted(sim_scores,
                        key=lambda x: x[1],
                        reverse=True)[:5]

    top_5_similarities[i] = [j[0] for j in sim_scores]

import pickle
pickle.dump(top_5_similarities, open('similarity.pkl','wb'))